# Decision Tress using `scikit-learn`

Scikit-learn provides a range of supervised and unsupervised learning algorithms via a consistent interface in Python. In this assigment you'll explore how to train decision trees using the `scikit-learn` library. The scikit-learn documentation can be found [here](http://scikit-learn.org/stable/documentation.html).

In this assignment we'll attempt to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains 1151 instances and 20 attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [1]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time

In [2]:
%matplotlib inline

In [3]:
# read in data 
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

data = pd.read_csv("messidor_features.txt", names = col_names)
print(data.shape)
data.head(10)

(1151, 20)


,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class,label
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1
5,1,1,44,43,41,41,37,29,28.356400,6.935636,2.305771,0.323724,0.000000,0.000000,0.000000,0.000000,0.502831,0.126741,0,1
6,1,0,29,29,29,27,25,16,15.448398,9.113819,1.633493,0.000000,0.000000,0.000000,0.000000,0.000000,0.541743,0.139575,0,1
7,1,1,6,6,6,6,2,1,20.679649,9.497786,1.223660,0.150382,0.000000,0.000000,0.000000,0.000000,0.576318,0.071071,1,0
8,1,1,22,21,18,15,13,10,66.691933,23.545543,6.151117,0.496372,0.000000,0.000000,0.000000,0.000000,0.500073,0.116793,0,1
9,1,1,79,75,73,71,64,47,22.141784,10.054384,0.874633,0.099780,0.023386,0.000000,0.000000,0.000000,0.560959,0.109134,0,1


### 1. Data Preprocessing  & Dimensionality Reduction with PCA

Q1. Separate the feature columns from the class label column. You should end up with two separate data frames - one that contains all of the feature values and one that contains the class labels.

In [4]:
# separate class labels and feature values 
lab_data = data["label"] # class labels
feat_data = data.drop(["label"], axis = 1) # feature values

print(lab_data.shape)  
print(feat_data.shape) 

(1151,)
(1151, 19)


Q2. Use `sklearn.preprocessing.StandardScaler` to standardize the dataset’s features (mean = 0 and variance = 1). 

In [5]:
# import 
from sklearn.preprocessing import StandardScaler

# standardize data 
scaled_feat = StandardScaler().fit_transform(feat_data) 

print("mean =", scaled_feat.mean()) # 0
print("std = ", scaled_feat.std()) # 1

mean = -2.8876256637559514e-17
std =  1.0


Q3 . Split your dataset into training and test sets (80% - 20% split). Use `sklearn.model_selection.train_test_split` to help you in this task. You should be working with your standardized features from here forward. Display how many records are in the training set and how many are in the test set.

In [6]:
# import 
from sklearn.model_selection import train_test_split

# splitting data test and train
lab_train, lab_test, feat_train, feat_test = train_test_split(lab_data, feat_data, test_size = 0.20)

# train and test records 
print("lab_train =",lab_train.shape, " lab_test =",lab_test.shape)
print("feat_train =",feat_train.shape, " feat_test =",feat_test.shape)

lab_train = (920,)  lab_test = (231,)
feat_train = (920, 19)  feat_test = (231, 19)


Q4. PCA is affected by the scale of the features that is why it is important to standardize the dataset first. The principle components generated by PCA are sensitive to the shape of the data in d-dimensional space. 
* Carry out a principal components analysis using `sklearn.decomposition.PCA` Note that you are fitting PCA on the **training set** only (NOT the test set).
* Use the `pca.explained_variance_ratio_` field to determine how many principal components are needed so that 95% variance is retained. 
* Reduce the PCA-transformed-dataset to this number of columns and you'll use the resulting dataset for subsequent tasks.

* Once the training set's dimensionality has been reduced with PCA, transform the **test set** to the principal component space that was created. (Do not fit a new PCA. Use the same one that was created with the training set.)

In [13]:
# import 
from sklearn.decomposition import PCA 

# fit PCA to the training set 
pca = PCA() 
pca_data = pca.fit_transform(feat_train) 

# columns needed to retain 95% variance 
var_exp = pca.explained_variance_ratio_
print("variance in each pc:", var_exp)

cum_var_exp = np.cumsum(var_exp)
print("\ncumulative variance:", cum_var_exp)

# find index of where cum_var goes over 95%
n_cols = 1 + np.argmax(cum_var_exp > 0.95) 
print("\nn_cols to keep:", n_cols)

# reduce training set 
feat_train_pca = pca_data[:, :n_cols]

# transforming the test set 
feat_test_pca = pca.transform(feat_test)[:, :n_cols]
print()
print(feat_train_pca.shape, feat_test_pca.shape)

variance in each pc: [6.35752128e-01 3.17000954e-01 3.21634418e-02 9.46061707e-03
 3.08193723e-03 9.91802852e-04 8.45936868e-04 3.00776693e-04
 1.72640147e-04 9.47663461e-05 7.15505258e-05 2.58869750e-05
 2.28506456e-05 1.06549231e-05 3.06076396e-06 6.01348683e-07
 2.44947121e-07 1.07640140e-07 4.10238481e-08]

cumulative variance: [0.63575213 0.95275308 0.98491652 0.99437714 0.99745908 0.99845088
 0.99929682 0.99959759 0.99977023 0.999865   0.99993655 0.99996244
 0.99998529 0.99999594 0.99999901 0.99999961 0.99999985 0.99999996
 1.        ]

n_cols to keep: 2

(920, 2) (231, 2)


### 2. Training Decision Trees in `scikit-learn`

Q5. Use `sklearn.tree.DecisionTreeClassifier` to fit a decision tree classifier on the training set. Use entropy as the split criterion. 

In [14]:
from sklearn import tree 

# create decision tree 
dr = tree.DecisionTreeClassifier(criterion = "entropy")

# train tree 
dr = dr.fit(feat_train, lab_train)

Q6. Now that the tree has been learned from the training data, we can run the test data through and predict classes for our test data. Use the `predict` method of `DecisionTreeClassifier` to classify the test data. Then use `sklearn.metrics.accuracy_score` to print out the accuracy of the classifier on the test set.

In [15]:
from sklearn.metrics import accuracy_score 

# test the tree w/ test data 
dr_pred = dr.predict(feat_test)
print("accuracy on test data:", (accuracy_score(lab_test, dr_pred)))

accuracy on test data: 0.6233766233766234


Q7. Note that the DecisionTree classifier has many parameters that can be set. Try tweaking parameters like split criterion, max_depth, min_impurity_decrease, min_samples_leaf, min_samples_split, etc. to see how they affect accuracy.

In [16]:
# decision trees w/ different max_depth
dr_2 = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = 5)
dr_3 = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = 10)
dr_4 = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = 15)

# train the trees
dr_2 = dr_2.fit(feat_data, lab_data)
dr_3 = dr_3.fit(feat_data, lab_data)
dr_4 = dr_4.fit(feat_data, lab_data)

# predict using test data 
dr_2_pred = dr_2.predict(feat_test) 
dr_3_pred = dr_3.predict(feat_test) 
dr_4_pred = dr_4.predict(feat_test) 

# accuracy increases with higher max depth 
print("accuracy on 05 depth test data:", (accuracy_score(lab_test, dr_2_pred)))
print("accuracy on 10 depth test data:", (accuracy_score(lab_test, dr_3_pred)))
print("accuracy on 15 depth test data:", (accuracy_score(lab_test, dr_4_pred)))

accuracy on 05 depth test data: 0.7229437229437229
accuracy on 10 depth test data: 0.8744588744588745
accuracy on 15 depth test data: 0.922077922077922


### 3. Using K-fold Cross Validation

You have now built a decision tree and tested it's accuracy using the "holdout" method. But as discussed in class, this is not sufficient for estimating generalization accuracy. Instead, we should use Cross Validation to get a better estimate of accuracy. You will do that next.

Q7. Use `sklearn.model_selection.cross_val_score` to perform 10-fold cross validation on your decision tree. Report the accuracy of the model. For this step, revert back to using the data before it underwent PCA. So you want to use the standardized data that came out of Q2. 

In [17]:
# import 
from sklearn.model_selection import cross_val_score 

# get cross validation score 
dr_cross_val = tree.DecisionTreeClassifier(criterion = "entropy")
print()
print(dr_cross_val)

# 10 fold cross validation 
scores = cross_val_score(dr_cross_val, feat_data, lab_data, cv = 10) 
print("scores:", scores)

print("\naccuracy:", scores.mean()*100) 


DecisionTreeClassifier(criterion='entropy')
scores: [0.61206897 0.59130435 0.67826087 0.6173913  0.62608696 0.63478261
 0.60869565 0.59130435 0.69565217 0.59130435]

accuracy: 62.468515742128936


Q8. Now we want to tune our model to use the best parameters to avoid overfitting to our training data. Grid search is an approach to parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid. 
* Use `sklearn.model_selection.GridSearchCV` to find the best `max_depth`, `max_features`, and `min_samples_leaf` for your tree. Use 'accuracy' for the scoring criteria.
* Try the values [5,10,15,20] for `max_depth` and `min_samples_leaf`. Try [5,10,15] for `max_features`. 
* Use a 5-fold cross validation. 
* Print out the best value for each of the tested parameters.
* Print out the accuracy of the model with these best values.

In [18]:
# import 
from sklearn.model_selection import GridSearchCV

# set up parameters 
params = {"max_depth": [5, 10, 15, 20], "min_samples_leaf": [5, 10, 15, 20], "max_features": [5, 10, 15]}

# create gridsearch
grid_search = GridSearchCV(dr_cross_val, params, cv = 5, scoring = "accuracy") 
grid_search.fit(feat_data, lab_data) 

# best parameters 
print("best parameters:", grid_search.best_params_)

# accuracy at best parameters 
print("accuracy:", grid_search.best_score_*100)

best parameters: {'max_depth': 20, 'max_features': 15, 'min_samples_leaf': 15}
accuracy: 64.64106907585168


Q9. To perform the nested cross-validation that we discussed in class, you'll now need to pass the `GridSearchCV` into a `cross_val_score`. 

What this does is: the `cross_val_score` splits the data in to train and test sets for the first fold, and it passes the train set into `GridSearchCV`. `GridSearchCV` then splits that set into train and validation sets for k number of folds (the inner CV loop). The hyper-parameters for which the average score over all inner iterations is best, is reported as the `best_params_`, `best_score_`, and `best_estimator_`(best decision tree). This best decision tree is then evaluated with the test set from the `cross_val_score` (the outer CV loop). And this whole thing is repeated for the remaining k folds of the `cross_val_score` (the outer CV loop). 

That is a lot of explanation for a very complex (but IMPORTANT) process, which can all be performed with a single line of code!

Be patient for this one to run. The nested cross-validation loop can take some time. An asterisk [\*] next to the cell indicates that it is still running.

Print the accuracy of your tuned, cross-validated model. This is the official accuracy that you would report for your model.

In [19]:
# get cross valiation score of the whole tree 
nested_score = cross_val_score(grid_search, feat_data, lab_data, cv = 5)

print("accuracy:", nested_score.mean() * 100)

accuracy: 61.859966120835686


Q10. Our accuracy rate isn't very good. We wouldn't want to use this model in the real world to actually diagnose patients because it is wrong about 40% of the time! What can we do to improve the accuracy of the model? Answer as a comment:

In [20]:
'''
In order to imporove the accuracy of the model, feature engineering is necessary in order to add or remove features.
'''

'\nIn order to imporove the accuracy of the model, feature engineering is necessary in order to add or remove features.\n'